1\. Listázzuk azon vendégek nevét, email-címét és felhasználói nevét, akik egynél többször foglaltak!

- Hagyjuk ki azokat a vendégeket, akik május hónapban születtek!

In [7]:
SELECT v.USERNEV, v.NEV, COUNT(UGYFEL_FK)
FROM Vendeg v JOIN Foglalas f ON v.USERNEV = f.UGYFEL_FK
GROUP BY v.USERNEV
HAVING COUNT(UGYFEL_FK) > 1

: Msg 8120, Level 16, State 1, Line 1
Column 'Vendeg.NEV' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause.

Total execution time: 00:00:00.001

2\. Listázzuk azon vendégek adatait, akik a legtöbb férőhelyes szobát (vagy szobákat) már lefoglalták!

- Csak klímás szoba jöhet számításba

In [9]:
SELECT *
FROM Vendeg v JOIN Foglalas f ON v.USERNEV = f.UGYFEL_FK
              JOIN Szoba sz ON f.SZOBA_FK = sz.SZOBA_ID
WHERE sz.KLIMAS LIKE 'i' AND sz.Ferohely IN 
(
    SELECT MAX(FEROHELY)
    FROM Szoba
)

(3 rows affected)

Total execution time: 00:00:00.011

USERNEV,NEV,EMAIL,SZAML_CIM,SZUL_DAT,FOGLALAS_PK,UGYFEL_FK,SZOBA_FK,METTOL,MEDDIG,FELNOTT_SZAM,GYERMEK_SZAM,SZOBA_ID,SZOBA_SZAMA,FEROHELY,POTAGY,KLIMAS,SZALLAS_FK
JOZSEFG,Gyuris József,józsef.gyuris@mail.hu,2660 Balassagyarmat Petőfi utca 1/2.,1975-05-26,852,JOZSEFG,24,2016-05-23,2016-05-25,1,0,24,8,4,1,i,3
PETERB,Berendi Péter,péter.berendi@mail.hu,3980 Sátoraljaújhely Vasút utca 4/10.,1969-01-01,1167,PETERB,24,2016-10-19,2016-10-20,3,1,24,8,4,1,i,3
tunde,Turcsik Tünde,tunde@gmail.com,7130 Tolna Fő út 122.,1974-02-12,1415,tunde,24,2017-01-03,2017-01-06,4,1,24,8,4,1,i,3


3\. Jelenítsük meg, hogy az egyes szálláshelyek szobáit esetén a legutolsó 5 foglalás átlagosan hány főre történt!

- A számításnál vegyük figyelembe a felnőtteket és a gyermekeket is
- A foglalás dátumánál a METTOL mezőt használjuk

(1 row affected)

Total execution time: 00:00:00.013

Kulonbozo_Ertekek_Szama,Minimum_Ertek,Maximum_Ertek,Atlag_Ertek,Not_Null_Szazalek,Null_Szazalek
4,1,4,2,100.000000000000,0.000000000000


4\. Készítsen lekérdezést, amely megjeleníti, hogy szálláshelyenként, azon belül a pótágyak száma szerint hány főre foglaltak összesen (felnőtt + gyermek)!

- Jelenjenek meg a részösszegek és a végösszeg is!

In [15]:
SELECT IIF(SZALLAS_NEV IS NULL, 'Végösszeg', SZALLAS_NEV),
       COALESCE(CAST(POTAGY AS VARCHAR), 'Összesen') AS 'Pótágyak száma',
       SUM(FELNOTT_SZAM + GYERMEK_SZAM) AS 'Fő'
FROM szallashely szh JOIN Szoba sz ON szh.SZALLAS_ID = sz.SZALLAS_FK
                     JOIN Foglalas f ON sz.SZOBA_ID = f.SZOBA_FK
GROUP BY ROLLUP(SZALLAS_NEV, POTAGY)
ORDER BY SZALLAS_NEV, POTAGY;


(41 rows affected)

Total execution time: 00:00:00.017

(No column name),Pótágyak száma,Fő
Végösszeg,Összesen,2490
Családi Ház,Összesen,52
Családi Ház,0,10
Családi Ház,1,31
Családi Ház,2,11
Fortuna Apartman,Összesen,106
Fortuna Apartman,0,18
Fortuna Apartman,1,52
Fortuna Apartman,2,36
Fortuna Panzió,Összesen,311


5\. A Szoba tábla Ferohely oszlopára végezzük el a következő számításokat:

- Különböző értékek száma
- Minimum, maximum, átlag
- Null és Not null értékek százalékos aránya

In [6]:
SELECT 
    COUNT(DISTINCT Ferohely) AS 'Különböző értékek száma',
    MIN(Ferohely) AS 'Minimum',
    MAX(Ferohely) AS 'Maximum',
    AVG(Ferohely) AS 'Átlag',
    (COUNT(Ferohely) * 100.0 / COUNT(*)) AS 'Nem nulla (%)',
    ((COUNT(*) - COUNT(Ferohely)) * 100.0 / COUNT(*)) AS 'Nulla (%)'
FROM Szoba

(1 row affected)

Total execution time: 00:00:00.007

Különböző értékek száma,Minimum,Maximum,Átlag,Nem nulla (%),Nulla (%)
4,1,4,2,100.000000000000,0.000000000000


6\. feladat

hset iro:1 nev 'Kiss Bela' nemzetiseg 'magyar'

hset iro:2 nev 'Nagy Istvan' nemzetiseg 'nemet'

hset konyv:1:1 cim 'Asd' mufaj 'novella'

hset konyv:2:2 cim 'Asd Asd' mufaj 'regeny'

7\. feladat

CREATE TABLE student (

id int primary key,

teljesitett\_felevek list\<int\>,

eredmenyek map\<int, double\>,

nyelvtudas set\<text\>

);

  

INSERT INTO student (id, eredmenyek, nyelvtudas, teljesitett\_felevek) values (

1,

{1:3.00},

{'magyar', 'nemet'},

\[1\]);

  

INSERT INTO student (id, eredmenyek, nyelvtudas, teljesitett\_felevek) values (

2,

{1:3.5, 2:4.8},

{'angol', 'francia', 'magyar'},

\[1, 2\]);